In [22]:
# train_dyn_v3.py
"""Train *from scratch* the v3 dynamics model for the new scenario
(two‑lap drift + recovery).  Works with the augmented dataset that already
contains `gas` and `is_drift`.

Usage (from repo root):

```bash
python train_dyn_v3.py  data/*.csv   \
                       --epochs 300  \
                       --batch 2048  \
                       --out dyn_v3.pt
```

Outputs:
    • NPZ file  (X, Y, mu*, sig*)
    • PyTorch checkpoint  dyn_v3.pt  (weights + μ/σ)

The model maps  (yawRate, ay_world, beta, speed, a_steer, a_gas)
            to  Δ(state)  for the next 20‑ms tick.
"""
from __future__ import annotations
import argparse, sys, math
from pathlib import Path

import pandas as pd
import numpy as np
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import glob, json, math, sys
from pathlib import Path

import numpy as np
import pandas as pd
import torch, torch.nn as nn, torch.optim as optim



In [23]:
################################################################################
# 1. CLI
################################################################################

CSV_GLOB   = "../new_data/circle/*.csv"   # could be list ["file1.csv", "file2.csv"]
EPOCHS     = 300
BATCH_SIZE = 2048
OUT_MODEL  = "dyn_v3.pt"
OUT_NPZ    = "model_dataset_v3.npz"


In [24]:
################################################################################
# 2. Feature engineering helpers
################################################################################
STEER_MIN, STEER_MAX = 1968, 4004
GAS_MIN,   GAS_MAX   = 2886, 4002 
STEER_C, STEER_SP = (STEER_MIN+STEER_MAX)/2, (STEER_MAX-STEER_MIN)/2
GAS_C,   GAS_SP   = (GAS_MIN+GAS_MAX)/2,   (GAS_MAX-GAS_MIN)/2

# PD‐like helper for numeric stability
EPS = 1e-6

def add_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add beta, speed, action normalisation."""
    # velocity components (m/s)   — 20‑ms delta
    dt = df["t_sec"].diff().fillna(0.02)
    df["vx"] = df["x_world"].diff() / dt
    df["vy"] = df["y_world"].diff() / dt
    df["speed"] = np.hypot(df["vx"], df["vy"])
    # beta = angle between velocity vector and yaw
    vel_ang = np.arctan2(df["vy"], df["vx"])
    df["beta"] = (vel_ang - df["yaw_rad"] + np.pi) % (2*np.pi) - np.pi   # wrap to [-π,π]
    # normalised actions
    df["a_steer"] = (df["steer"] - STEER_C) / STEER_SP
    df["a_gas"]   = (df["gas"]   - GAS_C)   / GAS_SP
    return df

def load_all(csv_glob: str) -> pd.DataFrame:
    files = [Path(f) for f in glob.glob(csv_glob)]
    if not files:
        sys.exit(f"No CSVs matched {csv_glob!r}")
    print(f"Found {len(files)} CSV files")
    return pd.concat((pd.read_csv(f) for f in files), ignore_index=True)

# world‑frame speed and beta
def add_speed_beta(df: pd.DataFrame) -> pd.DataFrame:
    vx = df["x_world"].diff() / df["t_sec"].diff()
    vy = df["y_world"].diff() / df["t_sec"].diff()
    df["vx"], df["vy"] = vx, vy
    df["speed"] = np.hypot(vx, vy)
    df["beta"]  = np.arctan2(vy, vx) - df["yaw_rad"]
    return df.dropna().reset_index(drop=True)

def build_matrices(df: pd.DataFrame):
    # state s_t
    S = df[["yawRate", "ay_world", "beta", "speed"]].values.astype(np.float32)
    # actions
    a_steer = ((df["steer"]-STEER_C)/STEER_SP).values.astype(np.float32)
    a_gas   = ((df["gas"]  -GAS_C)/GAS_SP  ).values.astype(np.float32)
    A = np.stack([a_steer, a_gas], axis=1)
    # drop last row to align with s_{t+1}
    S, A = S[:-1], A[:-1]
    Sn   = df[["yawRate", "ay_world", "beta", "speed"]].values.astype(np.float32)[1:]
    dS   = Sn - S
    X    = np.hstack([S, A])
    return X, dS


In [25]:
################################################################################
# 3. Neural net definition
################################################################################
class DynNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(6, 128), nn.ReLU(),
            nn.Linear(128, 128), nn.ReLU(),
            nn.Linear(128, 4)
        )
    def forward(self, x):
        return self.net(x)

In [26]:
################################################################################
# 4. Main training routine
################################################################################
print("\n>>> Loading CSVs …")
df_raw = load_all(CSV_GLOB)
print(">>> Feature engineering …")
df = add_speed_beta(df_raw.copy())
X, Y = build_matrices(df)
print(f"Dataset  X: {X.shape},  Y: {Y.shape}")

# Normalise
mu_X, sig_X = X.mean(0), X.std(0)+1e-6
mu_Y, sig_Y = Y.mean(0), Y.std(0)+1e-6
Xn = (X - mu_X) / sig_X
Yn = (Y - mu_Y) / sig_Y

np.savez(OUT_NPZ, X=Xn, Y=Yn, mu_X=mu_X, sig_X=sig_X, mu_Y=mu_Y, sig_Y=sig_Y)
print(f"Saved NPZ → {OUT_NPZ}")

# Torch tensors
X_t = torch.tensor(Xn)
Y_t = torch.tensor(Yn)

dataset = torch.utils.data.TensorDataset(X_t, Y_t)
loader  = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

net = DynNet()
opt = optim.Adam(net.parameters(), 1e-3)

print(">>> Training …")
for epoch in range(1, EPOCHS+1):
    running = 0.0
    for xb, yb in loader:
        pred = net(xb)
        loss = ((pred - yb)**2).mean()
        opt.zero_grad(); loss.backward(); opt.step()
        running += loss.item() * len(xb)
    if epoch % 20 == 0 or epoch == 1:
        mse = running / len(dataset)
        print(f"Epoch {epoch:3d}/{EPOCHS}   train‑MSE={mse:.4e}")

# Save
ckpt = {
    "net": net.state_dict(),
    "mu_X": mu_X, "sig_X": sig_X,
    "mu_Y": mu_Y, "sig_Y": sig_Y,
    "config": dict(CSV_GLOB=CSV_GLOB, EPOCHS=EPOCHS,
                    BATCH=BATCH_SIZE)
}
torch.save(ckpt, OUT_MODEL)
print(f"✓ Saved model → {OUT_MODEL}\nDone.")


>>> Loading CSVs …
Found 60 CSV files
>>> Feature engineering …
Dataset  X: (1921, 6),  Y: (1921, 4)
Saved NPZ → model_dataset_v3.npz
>>> Training …
Epoch   1/300   train‑MSE=1.0136e+00
Epoch  20/300   train‑MSE=7.8975e-01
Epoch  40/300   train‑MSE=7.4797e-01
Epoch  60/300   train‑MSE=7.2867e-01
Epoch  80/300   train‑MSE=7.1322e-01
Epoch 100/300   train‑MSE=6.9825e-01
Epoch 120/300   train‑MSE=6.8302e-01
Epoch 140/300   train‑MSE=6.6722e-01
Epoch 160/300   train‑MSE=6.5092e-01
Epoch 180/300   train‑MSE=6.3396e-01
Epoch 200/300   train‑MSE=6.1643e-01
Epoch 220/300   train‑MSE=5.9884e-01
Epoch 240/300   train‑MSE=5.8253e-01
Epoch 260/300   train‑MSE=5.6714e-01
Epoch 280/300   train‑MSE=5.5296e-01
Epoch 300/300   train‑MSE=5.3922e-01
✓ Saved model → dyn_v3.pt
Done.
